### Uber NYC Trip Data ETL

Direct import from Kaggle version of this dataset does not work because archive contains multiple files with different schema.  Turns out, complete dataset we want spread across several 'uber-raw-data' tables

In [94]:
local_storage = '/freenas-theExpanse/mflaxman/uber_nyc_benchmark'

#### For all raw data

Drop prior table to repeat.

In [96]:
if 0:
    con.con.execute("DROP TABLE IF EXISTS uber_nyc_trips")

Download and unzip Uber's data to a folder specified here

In [21]:
filespec = f"/freenas-theExpanse/mflaxman/uber_nyc_benchmark/uber-raw-data*.csv"

In [23]:
count = 0
pbar = tqdm(glob.glob(filespec))

for f in pbar:
    short_name = f.split('/')[-1]
    pbar.set_description(f"Processing {short_name}")
    count += 1
    if count == 1:
        ddl = "trip_datetime TIMESTAMP, latitude DOUBLE, longitude DOUBLE, base TEXT"
        con.con.execute(f"CREATE TABLE IF NOT EXISTS uber_nyc_trips ({ddl})")
    df = pd.read_csv(f) 
    if not 'Date/Time' in df.columns:
        pbar.set_description("Found CSV with unexpected scheme, skipping")
        continue
    # forward slashes in SQL column names won't do...rename more sensibly
    df.rename(columns={"Date/Time":"trip_datetime"},inplace=True)
    # force column type
    df.trip_datetime = pd.to_datetime(df.trip_datetime)
    try:
        con.con.load_table('uber_nyc_trips',df)
        pbar.set_description(f"loaded")
    except:
        pbar.set_description(f"failed to load, continuing")

loaded: 100%|██████████| 7/7 [04:47<00:00, 41.06s/it]                                                                            
